#### Imports

In [ ]:
from typing import Union
from typing_extensions import NotRequired, TypedDict
from functools import reduce

from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as mcolors
import numpy as np
import xarray as xr
import textwrap

from sdm_eurec4a.visulization import (
    adjust_lightness_array,
    set_custom_rcParams,
    handler_map_alpha,
    add_subplotlabel,
)
from sdm_eurec4a.reductions import mean_and_stderror_of_mean

from sdm_eurec4a import RepositoryPath
import warnings


warnings.filterwarnings("ignore")

In [ ]:
def set_xticks_time(ax):
    xticks = [0, 500, 1000]
    ax.set_xticks(xticks)


def set_yticks_height(ax):
    yticks = [0, 500, 1000, 1500, 2000]
    ax.set_yticks(yticks)


def set_yticks_height_km(ax):
    yticks = [0, 0.5, 1, 1.5, 2]
    ax.set_yticks(yticks)


def set_logxticks_meter(ax):
    xticks = [1e-6, 1e-3]
    xticklabels = [r"$10^{-6}$", r"$10^{-3}$"]
    ax.set_xticks(xticks, xticklabels)


def set_logxticks_micrometer(ax):
    xticks = [1e-3, 1e0, 1e3]
    xticklabels = [r"$10^{-3}$", r"$10^{0}$", r"$10^{3}$"]
    ax.set_xticks(xticks, xticklabels)


def set_logtyticks_psd(ax):
    yticks = [1e0, 1e6]
    yticklabels = [r"$10^0$", r"$10^6$"]
    ax.set_yticks(yticks, yticklabels)


def set_yticks_lwc(ax):
    ax.set_yticks([0, 0.1, 0.2])


def set_ticks_precipitation(ax, x=False, y=False):
    ticks = np.arange(0, 0.21, 0.05)
    if x == True:
        ax.set_xticks(ticks)
        ax.set_xlim(ticks.min(), ticks.max())
    if y == True:
        ax.set_yticks(ticks)
        ax.set_ylim(ticks.min(), ticks.max())

In [ ]:
def label_from_attrs(
    da: xr.DataArray,
    return_name: bool = True,
    return_units: bool = True,
    linebreak: bool = False,
    name_width: Union[None, int] = None,
    units_appendix: Union[None, str] = None,
) -> str:
    try:
        name = f"{da.attrs['long_name']}"
    except KeyError:
        name = f"{da.name}"

    if "units" in da.attrs:
        units = f"{da.attrs['units']}"
        if "$" not in units:
            units = f"${units}$"

        units = units.replace("$", " ")
    else:
        units = "???"

    if units_appendix != None:
        units = f"{units} {units_appendix}"

    # create latex string
    units = rf"$\left[ {units} \right]$"

    if return_name == True:
        if name_width == None:
            name = name
        else:
            name = textwrap.fill(name, name_width)

    if return_name == True and return_units == True:
        if linebreak == True:
            return f"{name}\n{units}"
        else:
            return f"{name} {units}"

    elif return_name == True and return_units == False:
        return f"{name}"
    elif return_name == False and return_units == True:
        return f"{units}"
    else:
        return ""

In [ ]:
import statsmodels.api as sm


def linear_fit(x: xr.DataArray, y: xr.DataArray):
    x = x.values.flatten()
    y = y.values.flatten()
    idx = np.argsort(x)
    x = x[idx]
    y = y[idx]
    idx = np.isfinite(x) & np.isfinite(y)
    x = x[idx]
    y = y[idx]
    X = np.column_stack((x,))
    X = sm.add_constant(X)

    corr = np.corrcoef(x, y)

    model = sm.OLS(y, X)
    results = model.fit()

    return results, corr, x, y


def linear_fit_new(x: Union[np.ndarray, xr.DataArray], y: Union[np.ndarray, xr.DataArray]):
    if isinstance(x, xr.DataArray):
        x = x.values.flatten()
    elif isinstance(x, np.ndarray):
        x = x.flatten()
    else:
        raise ValueError("x and y must be either xr.DataArray or np.ndarray")
    if isinstance(y, xr.DataArray):
        y = y.values.flatten()
    elif isinstance(y, np.ndarray):
        y = y.flatten()
    else:
        raise ValueError("x and y must be either xr.DataArray or np.ndarray")

    idx = np.argsort(x)
    x = x[idx]
    y = y[idx]
    idx = np.isfinite(x) & np.isfinite(y)
    x = x[idx]
    y = y[idx]
    X = np.column_stack((x,))
    X = sm.add_constant(X)

    corr = np.corrcoef(x, y)

    model = sm.OLS(y, X)
    results = model.fit()

    return results, corr


def linear_fit_plot(ax, x: xr.DataArray, y: xr.DataArray, alpha: float = 0.05):
    results, corr, x, y = linear_fit(x, y)

    pred_ols = results.get_prediction()
    iv_l = pred_ols.summary_frame(alpha=alpha)["obs_ci_lower"]
    iv_u = pred_ols.summary_frame(alpha=alpha)["obs_ci_upper"]

    ax.plot(x, results.fittedvalues, "r-", label="OLS fit ($R$= {:.2f})".format(corr[0, 1]))
    ax.plot(x, iv_u, "r:", label=f"{(1-alpha)*100:.0f}$\\%$ CI")
    ax.plot(x, iv_l, "r:")

In [ ]:
plt.style.use("default")
default_colors = set_custom_rcParams()
plt.rcParams.update(
    {
        "axes.spines.top": False,
        "axes.spines.right": False,
        "axes.spines.left": False,
        "axes.spines.bottom": False,
    }
)

dark_colors = adjust_lightness_array(default_colors, amount=0.5)

repo_path = RepositoryPath("levante")()
print(repo_path)

subdata_dir = "output_v3.5"
data_path = Path("/home/m/m301096/CLEO/data/") / subdata_dir

ds_subpath = "combined/eulerian_dataset_combined_v2.nc"

# THE PATH TO THE SCRIPT DIRECTORY
script_dir = Path("/home/m/m301096/repositories/sdm-eurec4a/notebooks/thesis/results/")
print(script_dir)


fig_dir = repo_path / "results" / script_dir.relative_to(repo_path) / subdata_dir / "stationary"
print(fig_dir)
fig_dir.mkdir(parents=True, exist_ok=True)


microphysics = (
    "null_microphysics",
    "condensation",
    "collision_condensation",
    "coalbure_condensation_large",
)

/home/m/m301096/repositories/sdm-eurec4a
/home/m/m301096/repositories/sdm-eurec4a/notebooks/thesis/results
/home/m/m301096/repositories/sdm-eurec4a/results/notebooks/thesis/results/output_v3.5/stationary


# Set time slice to use for temporal mean and median

In [ ]:
time_slice = slice(1500, 3590)  # seconds
radius_split = 45  # µm
radius_slice = slice(1e0, None)  # µm

# Load datasets

In [ ]:
class MicrophysicDict(TypedDict):
    dataset: xr.Dataset
    microphysics: str
    path: Path
    # linestyle: Union[str, tuple]
    color: str


class OptionalDictOfMicrophysicDict(TypedDict):
    null_microphysics: NotRequired[MicrophysicDict]
    condensation: NotRequired[MicrophysicDict]
    collision_condensation: NotRequired[MicrophysicDict]
    coalbure_condensation_large: NotRequired[MicrophysicDict]
    coalbure_condensation_small: NotRequired[MicrophysicDict]


class DictOfMicrophysicDict(TypedDict):
    null_microphysics: MicrophysicDict
    condensation: MicrophysicDict
    collision_condensation: MicrophysicDict
    coalbure_condensation_cke: MicrophysicDict
    coalbure_condensation_large: MicrophysicDict
    coalbure_condensation_small: MicrophysicDict


data_dict = DictOfMicrophysicDict(
    null_microphysics=MicrophysicDict(
        microphysics="Null microphysics",
        path=Path(),
        dataset=xr.Dataset(),
        color="k",
    ),
    condensation=MicrophysicDict(
        microphysics="Condensation/Evaporation",
        path=Path(),
        dataset=xr.Dataset(),
        color="k",
    ),
    collision_condensation=MicrophysicDict(
        microphysics="Coll-coal, cond./evap.",
        path=Path(),
        dataset=xr.Dataset(),
        color="k",
    ),
    coalbure_condensation_cke=MicrophysicDict(
        microphysics="Coll-coal-breakup n by CKE\nand cond./evap.",
        path=Path(),
        dataset=xr.Dataset(),
        color="k",
    ),
    coalbure_condensation_large=MicrophysicDict(
        microphysics="Coll-coal-breakup (n=125)\nand cond./evap.",
        path=Path(),
        dataset=xr.Dataset(),
        color="k",
    ),
    coalbure_condensation_small=MicrophysicDict(
        microphysics="Coll-coal-breakup (n=5)\nand cond./evap.",
        path=Path(),
        dataset=xr.Dataset(),
        color="k",
    ),
)

colors_dict = dict(
    null_microphysics="grey",
    condensation=default_colors[0],
    collision_condensation=default_colors[1],
    coalbure_condensation_cke=default_colors[2],
    coalbure_condensation_large=default_colors[3],
    coalbure_condensation_small=default_colors[4],
)

for mp in data_dict:
    data_dict[mp]["path"] = data_path / f"{mp}" / ds_subpath
    data_dict[mp]["color"] = colors_dict[mp]

for key in data_dict:
    ds = xr.open_dataset(data_dict[key]["path"], chunks={"cloud_id": 2})
    # ds = ds.sel(cloud_id = [18, 301])
    ds.attrs.update(microphysics=data_dict[key]["microphysics"])
    ds.attrs.update(microphysics_short=key)

    data_dict[key]["dataset"] = ds

# ---------------------------------------------------- #
# Reindex the datasets to have the same radius bins
# ---------------------------------------------------- #

combined_radius_bins = reduce(
    np.union1d, [data_dict[mp]["dataset"]["radius_bins"].values for mp in data_dict]
)
fill_value = np.nan
print("Number of radius bins:", len(combined_radius_bins))
print("Fill value:", fill_value)
for mp in data_dict:
    data_dict[mp]["dataset"] = data_dict[mp]["dataset"].reindex(
        radius_bins=combined_radius_bins, fill_value=fill_value
    )

Number of radius bins: 76
Fill value: nan


### All clouds which are simulated

In [ ]:
intersect_cloud_ids = reduce(
    np.intersect1d, [data_dict[key]["dataset"]["cloud_id"].data for key in data_dict]
)
for mp in data_dict:
    print(mp, len(data_dict[mp]["dataset"]["cloud_id"]))

print("\nIntersect:", len(intersect_cloud_ids))
print("cloud_ids:", intersect_cloud_ids)

null_microphysics 94
condensation 78
collision_condensation 83
coalbure_condensation_cke 91
coalbure_condensation_large 93
coalbure_condensation_small 94

Intersect: 66
cloud_ids: [  9  11  18  20  21  22  65  67  68  71  72  73  74  88  94 110 113 114
 130 135 136 142 194 197 198 199 201 203 205 207 208 211 212 213 214 215
 217 218 219 220 221 222 223 224 230 233 235 236 237 292 293 295 296 301
 303 305 306 307 308 309 311 312 314 359 361 362]


In [ ]:
clouds_dict = {
    "222": dict(
        cloud_id=222,
        color="r",
    ),
    "142": dict(
        cloud_id=142,
        color="b",
    ),
}
for key in data_dict:
    ds = data_dict[key]["dataset"]
    for cloud_id in clouds_dict:
        cloud_id = clouds_dict[cloud_id]["cloud_id"]
        is_in = cloud_id in ds["cloud_id"]
        print(f"{cloud_id}, {key}, {is_in}")

222, null_microphysics, True
142, null_microphysics, True
222, condensation, True
142, condensation, True
222, collision_condensation, True
142, collision_condensation, True
222, coalbure_condensation_cke, True
142, coalbure_condensation_cke, True
222, coalbure_condensation_large, True
142, coalbure_condensation_large, True
222, coalbure_condensation_small, True
142, coalbure_condensation_small, True


### Add further variables e.g. latent cooling

In [ ]:
# NOTE: For now, it needs to be divided by 2 s to get values per second Bue to a bug.
def add_variables(
    ds: xr.Dataset, latent_heat_of_condensation: float = 2.265e6, time_slice=time_slice  # J kg-1
):
    # fix attributes from basic variables
    ds["mass_represented"].attrs["long_name"] = "Mass"

    ds["radius_bins"].attrs["long_name"] = "Radius"
    ds["radius_bins"].attrs["description"] = "Bin centers of the radius bins"
    ds["radius_bins"].attrs["units"] = "$\\mu m$"

    ds["relative_humidity"].attrs["long_name"] = "Relative humidity"
    ds["relative_humidity"].attrs["units"] = "$\\%$"

    ds["gridbox_thickness"] = ds["gridbox_top"] - ds["gridbox_bottom"]
    ds["gridbox_thickness"].attrs["long_name"] = "Gridbox thickness"
    ds["gridbox_thickness"].attrs["units"] = "$m$"

    ds["cloud_base_height"] = ds["gridbox_coord3"].sel(gridbox=ds["max_gridbox"])
    ds["cloud_base_height"].attrs["long_name"] = "Cloud base height"
    ds["cloud_base_height"].attrs["units"] = "$m$"

    # It seems that xi was stored as an integer. This is not wanted, because nan values will just be large integers.
    ds["xi"] = ds["xi"].astype(float)
    ds["xi"] = ds["xi"].where(ds["xi"] < 1e12)
    ds["xi"].attrs["units"] = ""
    ds["xi"].attrs["long_name"] = "Real droplet num. conc."
    ds["xi"].attrs["description"] = "Real droplet number concentration in the gridbox"

    ds["xi_per_volume"] = ds["xi"] / ds["gridbox_volume"]
    ds["xi_per_volume"].attrs["long_name"] = ds["xi"].attrs["long_name"]
    ds["xi_per_volume"].attrs["units"] = "$m^{-3}$"
    ds["xi_per_volume"].attrs[
        "description"
    ] = "Real droplet number concentration in one cubic meter of air"

    ds["number_superdroplets_per_volume"] = 1000 * ds["number_superdroplets"] / ds["gridbox_volume"]
    ds["number_superdroplets_per_volume"].attrs["units"] = "$10^3m^{-3}$"
    ds["number_superdroplets_per_volume"].attrs[
        "long_name"
    ] = "Number of superdroplets in 1000 cubic meter of air"

    ds["mass_represented_per_volume"] = 1e3 * ds["mass_represented"] / ds["gridbox_volume"]
    ds["mass_represented_per_volume"].attrs["units"] = "$g m^{-3}$"
    ds["mass_represented_per_volume"].attrs["long_name"] = "Mass"
    ds["mass_represented_per_volume"].attrs[
        "description"
    ] = "Mass of the represented droplets in one cubic meter of air"

    ds["massdelta_condensation_per_volume"] = ds["massdelta_condensation"] / ds["gridbox_volume"]
    ds["massdelta_condensation_per_volume"].attrs["units"] = "$kg m^{-3} s^{-1}$"
    ds["massdelta_condensation_per_volume"].attrs["long_name"] = "Condensation rate"
    ds["massdelta_condensation_per_volume"].attrs[
        "description"
    ] = "Condensation rate in one cubic meter of air"

    ds["latent_cooling_full"] = (
        -ds["massdelta_condensation"].sel(time=time_slice) * latent_heat_of_condensation
    )  # kg m-3 s-1 * J kg-1 = W m-3
    ds["latent_cooling_full"].attrs["units"] = "$W m^{-3}$"
    ds["latent_cooling_full"].attrs["long_name"] = "Latent cooling"
    ds["latent_cooling_full"].attrs["description"] = "Latent cooling for all timesteps"

    ds["latent_cooling"] = (
        -ds["massdelta_condensation"]
        .sel(time=time_slice)
        .mean("time", keep_attrs=True, skipna=True)
        .where(ds["sub_cloud_layer_mask"])
        * latent_heat_of_condensation
    )
    ds["latent_cooling"].attrs["units"] = "$W m^{-3}$"  # kg m-3 s-1 * J kg-1 = W m-3
    ds["latent_cooling"].attrs["long_name"] = "Latent cooling"
    ds["latent_cooling"].attrs[
        "description"
    ] = "Latent cooling for the sub cloud layer. Calculated with the condensation monitor of CLEO."

    ds["latent_cooling_radius_bins"] = (
        -ds["mass_difference_per_volume"].sel(time=time_slice).where(ds["sub_cloud_layer_mask"])
        * latent_heat_of_condensation
    )
    ds["latent_cooling_radius_bins"].attrs["units"] = "$W m^{-3}$"  # kg m-3 s-1 * J kg-1 = W m-3
    ds["latent_cooling_radius_bins"].attrs["long_name"] = "Latent cooling"
    ds["latent_cooling_radius_bins"].attrs[
        "description"
    ] = "Latent cooling for the sub cloud layer. Calculated with the mass difference from lagrangian particles."


for mp in data_dict:
    ds = data_dict[mp]["dataset"]
    add_variables(ds)

## Identify outliers


It seems the outlier is cloud ``296``

In [ ]:
null_microphysics: xr.Dataset = data_dict["null_microphysics"]["dataset"]
condensation: xr.Dataset = data_dict["condensation"]["dataset"]
collision_condensation: xr.Dataset = data_dict["collision_condensation"]["dataset"]
coalbure_condensation_cke: xr.Dataset = data_dict["coalbure_condensation_cke"]["dataset"]
coalbure_condensation_large: xr.Dataset = data_dict["coalbure_condensation_large"]["dataset"]
coalbure_condensation_small: xr.Dataset = data_dict["coalbure_condensation_small"]["dataset"]

# fig, axs = plt.subplots(ncols=5, figsize=(15, 5))
# null_microphysics["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[0])
# condensation["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[1])
# collision_condensation["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[2])
# coalbure_condensation_large["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[3])
# coalbure_condensation_small["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[4])

# remove outlier cloud_id 296
cloud_id_selection = intersect_cloud_ids[intersect_cloud_ids != 296]

null_microphysics = null_microphysics.sel(cloud_id=cloud_id_selection)
condensation = condensation.sel(cloud_id=cloud_id_selection)
collision_condensation = collision_condensation.sel(cloud_id=cloud_id_selection)
coalbure_condensation_cke = coalbure_condensation_cke.sel(cloud_id=cloud_id_selection)
coalbure_condensation_large = coalbure_condensation_large.sel(cloud_id=cloud_id_selection)
coalbure_condensation_small = coalbure_condensation_small.sel(cloud_id=cloud_id_selection)

# Input Thermodynamic profiles and PSD

``NOTE``

We can state that the **median** of the lagrangian method gives reasonable results compared to tghe direct output from the evaporation monitor of CLEO.

### Thermodynamic profiles

In [ ]:
# assert that all relative humidities are the same for all microphysics
# use the combination of all datasets to check this
import itertools

mps = data_dict.keys()
all_combinations = list(itertools.combinations(mps, r=2))

for mp1, mp2 in all_combinations:
    print(mp1, mp2)
    ds1 = data_dict[mp1]["dataset"].sel(cloud_id=cloud_id_selection)
    ds2 = data_dict[mp2]["dataset"].sel(cloud_id=cloud_id_selection)
    xr.testing.assert_equal(ds1["relative_humidity"], ds2["relative_humidity"])

null_microphysics condensation
null_microphysics collision_condensation
null_microphysics coalbure_condensation_cke
null_microphysics coalbure_condensation_large
null_microphysics coalbure_condensation_small
condensation collision_condensation
condensation coalbure_condensation_cke
condensation coalbure_condensation_large
condensation coalbure_condensation_small
collision_condensation coalbure_condensation_cke
collision_condensation coalbure_condensation_large
collision_condensation coalbure_condensation_small
coalbure_condensation_cke coalbure_condensation_large
coalbure_condensation_cke coalbure_condensation_small
coalbure_condensation_large coalbure_condensation_small


## LWC comparison CLEO and ATR

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3.5))
ax.plot(
    null_microphysics["relative_humidity"].T,
    null_microphysics["gridbox_coord3"].T,
    alpha=0.4,
    color=dark_colors[6],
    linewidth=1,
)
# ax.set_title("Relative humidity profiles of all clouds")
ax.set_xlabel(label_from_attrs(condensation["relative_humidity"]))
ax.set_ylabel("Height [$m$]")

fig.tight_layout()
fig.savefig(bbox_inches="tight", fname=fig_dir / f"relative_humidity_profiles.svg")
fig.savefig(bbox_inches="tight", fname=fig_dir / f"relative_humidity_profiles.pdf")

#### PSD in cloud layer and near cloud base

In [ ]:
data = null_microphysics
psd = data["xi_per_volume"].sel(gridbox=data["max_gridbox"]).sel(time=time_slice).mean("time").compute()
fig, ax = plt.subplots(figsize=(5, 3.5))
ax.plot(
    psd["radius_bins"].expand_dims(cloud_id=psd["cloud_id"]).T,
    psd.T,
    alpha=0.6,
    color="grey",
    linewidth=1,
)
ax.set_xscale("log")
ax.set_yscale("symlog", linthresh=1e-3, linscale=0.2)
# ax.set_title("Droplet Size Distribution in cloud layer\nAfter one simulation timestep")
ax.set_ylabel(
    label_from_attrs(data["xi_per_volume"], units_appendix=" (\\log\\mu m)^{-1}", linebreak=True)
)
ax.set_xlabel(label_from_attrs(data["radius_bins"]))
ax.set_xlim(4e-2, 1.5e3)
ax.set_yticks(10.0 ** (np.arange(-3, 7, 3)))
ax.set_ylim(0, 5e7)

ax.axvline(45, color="k", linestyle="--")

fig.tight_layout()
fig.savefig(bbox_inches="tight", fname=fig_dir / f"particle_size_distribution_max.svg")
fig.savefig(bbox_inches="tight", fname=fig_dir / f"particle_size_distribution_max.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3.5))

data = null_microphysics
psd = (
    data["xi_per_volume"]
    .sel(gridbox=data["max_gridbox"])
    .sel(time=time_slice)
    .fillna(0)
    .mean("time")
    .compute()
)
ax.plot(
    psd["radius_bins"].expand_dims(cloud_id=psd["cloud_id"]).T,
    psd.T,
    alpha=0.2,
    color="grey",
    linewidth=1,
)


ax.set_xscale("log")
# ax.set_title("Droplet Size Distribution in cloud layer\nAfter one simulation timestep")
ax.set_ylabel(
    label_from_attrs(data["xi_per_volume"], units_appendix=" (\\log\\mu m)^{-1}", linebreak=True)
)
ax.set_xlabel(label_from_attrs(data["radius_bins"]))
ax.set_xlim(4e-2, 1.5e3)

ax.set_yscale("symlog", linthresh=1e-3, linscale=0.2)

ax.set_yticks(10.0 ** (np.arange(-3, 7, 3)))
ax.set_ylim(1e-1, 5e7)

# ax.set_ylim(0, 120)

ax.axvline(45, color="k", linestyle="--")

fig.tight_layout()
fig.savefig(bbox_inches="tight", fname=fig_dir / f"real-number-concentration-cloud_layer.pdf")


data = condensation
psd = (
    data["xi_per_volume"]
    .sel(gridbox=data["max_gridbox"] - 2)
    .sel(time=time_slice)
    .fillna(0)
    .mean("time")
    .compute()
)
ax.plot(
    psd["radius_bins"].expand_dims(cloud_id=psd["cloud_id"]).T,
    psd.T,
    alpha=0.2,
    color=dark_colors[6],
    linewidth=1,
)

fig.savefig(bbox_inches="tight", fname=fig_dir / f"real-number-concentration-comparison.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3.5))

# data = null_microphysics
# psd = data["xi_per_volume"].sel(gridbox=data["max_gridbox"]).sel(time=time_slice).fillna(0).mean("time").compute()
# ax.plot(
#     psd["radius_bins"].expand_dims(cloud_id=psd["cloud_id"]).T,
#     psd.T,
#     alpha=0.2,
#     color = "grey",
# )

data = condensation
psd = (
    data["xi_per_volume"]
    .sel(gridbox=data["max_gridbox"] - 2)
    .sel(time=time_slice)
    .fillna(0)
    .mean("time")
    .compute()
)
ax.plot(
    psd["radius_bins"].expand_dims(cloud_id=psd["cloud_id"]).T,
    psd.T,
    alpha=0.2,
    color=dark_colors[6],
    linewidth=1,
)

ax.set_xscale("log")
# ax.set_title("Droplet Size Distribution in cloud layer\nAfter one simulation timestep")
ax.set_ylabel(
    label_from_attrs(data["xi_per_volume"], units_appendix=" (\\log\\mu m)^{-1}", linebreak=True)
)
ax.set_xlabel(label_from_attrs(data["radius_bins"]))
ax.set_xlim(1e0, 1.5e3)

# ax.set_yscale("symlog", linthresh=1e-3, linscale=0.2)
# ax.set_yticks(10.0 ** (np.arange(-3, 7, 3)))
# ax.set_ylim(0, 5e7)

ax.set_ylim(0, 120)

ax.axvline(45, color="k", linestyle="--")

fig.tight_layout()
fig.savefig(bbox_inches="tight", fname=fig_dir / f"real-number-concentration-cloud_base.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3.5))

msd = (
    null_microphysics["mass_represented_per_volume"]
    .sel(gridbox=null_microphysics["max_gridbox"])
    .sel(time=time_slice)
    .mean("time")
)
ax.plot(
    msd["radius_bins"].expand_dims(cloud_id=psd["cloud_id"]).T,
    msd.T,
    alpha=0.2,
    color="grey",
)


msd = (
    condensation["mass_represented_per_volume"]
    .sel(gridbox=condensation["max_gridbox"] - 2)
    .sel(time=time_slice)
    .mean("time")
)
ax.plot(
    msd["radius_bins"].expand_dims(cloud_id=psd["cloud_id"]).T,
    msd.T,
    alpha=0.2,
    color=dark_colors[6],
)
ax.set_xscale("log")
ax.set_yscale("symlog", linthresh=1e-18, linscale=0.2)
# ax.set_title("Mass Distribution in cloud layer\nAfter one simulation timestep")
ax.set_xlabel(label_from_attrs(data["radius_bins"]))
ax.set_ylabel(
    label_from_attrs(condensation["mass_represented_per_volume"], units_appendix=" (\\log\\mu m)^{-1}")
)
ax.set_xlim(4e-2, 1.5e3)
ax.set_yticks(10.0 ** (np.arange(-18, 1, 3)))
ax.set_ylim(1e-10, 1e0)

ax.axvline(45, color="k", linestyle="--")

fig.tight_layout()
fig.savefig(bbox_inches="tight", fname=fig_dir / f"mass-size-distribution-comparison.svg")
fig.savefig(bbox_inches="tight", fname=fig_dir / f"mass-size-distribution-comparison.pdf")

# Precipitation

Non smoothed precipitation shows, that individual SDs are responsible for the surface precipiation.

Smoothing the timeseries gives chance to get a more robust value of the mean rain rate.

In [ ]:
data = condensation

surface_prec = data["precipitation"].sum(dim="radius_bins").chunk(dict(cloud_id=-1))

surface_prec_cloud_mean, surface_prec_cloud_sem = mean_and_stderror_of_mean(
    data=surface_prec,
    dims=("cloud_id",),
)

surface_prec_smooth = surface_prec.rolling(time=30, center=True).mean()
surface_prec_cloud_mean_smooth = surface_prec_cloud_mean.rolling(time=30, center=True).mean()
surface_prec_cloud_sem_smooth = surface_prec_cloud_sem.rolling(time=30, center=True).mean()

print("old dims", data["precipitation"].attrs["units"])
print("new dim", data["precipitation"].attrs["units"])
surface_prec.attrs["units"] = data["precipitation"].attrs["units"]
surface_prec.attrs["long_name"] = data["precipitation"].attrs["long_name"]

old dims mm/h
new dim mm/h


#### Histogram of Precipiation and its relation to the mass in the surface gridbox

In [ ]:
ylim = [0, 12]
xbins = np.arange(0, 0.21, 0.01)

precip_time_mean, precip_time_sem = mean_and_stderror_of_mean(
    data=surface_prec.sel(time=time_slice),
    dims=("time",),
)

precip_time_mean = precip_time_mean.compute()
precip_time_mean.attrs.update(**surface_prec.attrs)
precip_time_sem = precip_time_sem.compute()

precip_cloud_mean, precip_cloud_sem = mean_and_stderror_of_mean(
    data=precip_time_mean,
    dims=("cloud_id",),
    data_std=precip_time_sem,
)

fig, ax = plt.subplots(figsize=(7, 4.5))
ax.hist(
    precip_time_mean,
    bins=20,
    color="k",
    alpha=0.5,
    density=False,
)
ax.axvline(
    precip_cloud_mean,
    color="darkorange",
    label=f"Mean: {precip_cloud_mean.values:.3f} $\\pm$ {precip_cloud_sem.values:.3f} "
    + f"{surface_prec.attrs['units']}",
)

ax.fill_betweenx(
    ylim,
    precip_cloud_mean - 2 * precip_cloud_sem,
    precip_cloud_mean + 2 * precip_cloud_sem,
    color="orange",
    alpha=0.3,
    label="2 SEM",
)


ax.set_xlabel(label_from_attrs(precip_time_mean))
ax.set_ylabel("Occurence")
ax.legend(loc="upper left")
ax.set_ylim(ylim)

set_ticks_precipitation(ax, x=True)

fig.savefig(bbox_inches="tight", fname=fig_dir / "histogram_surface_precipitation.svg")
fig.savefig(bbox_inches="tight", fname=fig_dir / "histogram_surface_precipitation.pdf")

### Temporal and histogram 

In [ ]:
fig, ax = plt.subplots(figsize=(7, 3.5))

sp = surface_prec.rolling(time=30, center=True).mean()

precip_cloud_mean, precip_cloud_sem = mean_and_stderror_of_mean(
    data=sp,
    dims=("cloud_id",),
)
precip_time_mean, precip_time_sem = mean_and_stderror_of_mean(
    data=sp,
    dims=("time",),
)

m, sem = mean_and_stderror_of_mean(
    data=sp,
    dims=(
        "time",
        "cloud_id",
    ),
)


ax.plot(
    sp["time"],
    sp.T,
    color="grey",  # [0.75, 0.75, 0.75],
    linestyle="-",
    # marker = ".",
    alpha=0.1,
    zorder=0,
)
ax.plot(
    np.nan,
    np.nan,
    color="grey",  # [0.75, 0.75, 0.75],
    linestyle="-",
    # marker = ".",
    alpha=0.1,
    zorder=0,
    label="Individual clouds",
)

ax.plot(
    precip_cloud_mean["time"],
    precip_cloud_mean.T,
    color="k",  # [0.75, 0.75, 0.75],
    linestyle="-",
    alpha=1,
    zorder=3,
    label="Mean over clouds",
)

ax.fill_between(
    precip_cloud_mean["time"],
    (precip_cloud_mean - 2 * precip_cloud_sem).T,
    (precip_cloud_mean + 2 * precip_cloud_sem).T,
    color="k",
    alpha=0.5,
    zorder=2,
    label="2 SEM",
)

ax.hlines(
    m,
    0,
    3590,
    color="darkorange",
    linestyle="-",
    linewidth=2,
    alpha=1,
    zorder=4,
    label="Total mean",
)

ax.fill_between(
    [0, 3600],
    [m - 2 * sem, m - 2 * sem],
    [m + 2 * sem, m + 2 * sem],
    color="orange",
    alpha=0.5,
    zorder=3,
    label="2 SEM",
)

ax.vlines(
    x=(time_slice.start, time_slice.stop - 15),
    ymin=0,
    ymax=0.4,
    color="r",
    linestyle="--",
    label="Stationary state",
    zorder=6,
)


ax.set_xlim(0, time_slice.stop - 5)
ax.set_ylim(0, 0.3)
ax.set_yticks([0, 0.1, 0.2, 0.3])

ax.set_xlabel("Time [s]")
ax.set_ylabel(label_from_attrs(sp))

ax.legend(handler_map=handler_map_alpha())
fig.tight_layout()
fig.savefig(bbox_inches="tight", fname=fig_dir / f"precipitation_temporal.svg")
fig.savefig(bbox_inches="tight", fname=fig_dir / f"precipitation_temporal.pdf")

### Origin of Frequency

In [ ]:
sp = surface_prec.rolling(time=30, center=True).mean()

precip_cloud_mean, precip_cloud_sem = mean_and_stderror_of_mean(
    data=sp,
    dims=("cloud_id",),
)
precip_time_mean, precip_time_sem = mean_and_stderror_of_mean(
    data=sp,
    dims=("time",),
)

m, sem = mean_and_stderror_of_mean(
    data=sp,
    dims=(
        "time",
        "cloud_id",
    ),
)


fig, ax = plt.subplots(figsize=(10, 4.5))

ax.plot(
    sp["time"],
    sp.T,
    color="grey",  # [0.75, 0.75, 0.75],
    linestyle="-",
    # marker = ".",
    alpha=0.1,
    zorder=0,
)
ax.plot(
    np.nan,
    np.nan,
    color="grey",  # [0.75, 0.75, 0.75],
    linestyle="-",
    # marker = ".",
    alpha=0.1,
    zorder=0,
    label="Individual clouds",
)

ax.plot(
    precip_cloud_mean["time"],
    precip_cloud_mean.T,
    color="k",  # [0.75, 0.75, 0.75],
    linestyle="-",
    alpha=1,
    zorder=3,
    label="Mean over clouds",
)

ax.fill_between(
    precip_cloud_mean["time"],
    (precip_cloud_mean - 2 * precip_cloud_sem).T,
    (precip_cloud_mean + 2 * precip_cloud_sem).T,
    color="k",
    alpha=0.5,
    zorder=2,
    label="2 SEM",
)

ax.hlines(
    m,
    0,
    3600,
    color="darkorange",
    linestyle="-",
    linewidth=1,
    alpha=1,
    zorder=4,
    label="Total mean",
)

ax.fill_between(
    [0, 3600],
    [m - 2 * sem, m - 2 * sem],
    [m + 2 * sem, m + 2 * sem],
    color="orange",
    alpha=0.5,
    zorder=3,
    label="2 SEM",
)

cloud_ids = [18, 142]
for color, cloud_id in zip(("r", "b"), cloud_ids):
    ax.plot(
        sp["time"],
        sp.sel(cloud_id=cloud_id).T,
        linestyle="-",
        # marker = ".",
        color=color,
        alpha=0.75,
        zorder=1,
        label=f"Cloud {cloud_id}",
    )


ax.set_xlim(0, 3600)
ax.set_ylim(0, 0.4)

ax.set_xlabel("Time [s]")
ax.set_ylabel(label_from_attrs(sp))

ax.legend(handler_map=handler_map_alpha())

fig.tight_layout()
fig.savefig(bbox_inches="tight", fname=fig_dir / f"precipitation_temporal_and_individual_clouds.svg")
fig.savefig(bbox_inches="tight", fname=fig_dir / f"precipitation_temporal_and_individual_clouds.pdf")

In [ ]:
sp = surface_prec.sel(time=slice(0, 300))  # .rolling(time=30, center=True).mean()

precip_cloud_mean, precip_cloud_sem = mean_and_stderror_of_mean(
    data=sp,
    dims=("cloud_id",),
)
precip_time_mean, precip_time_sem = mean_and_stderror_of_mean(
    data=sp,
    dims=("time",),
)

m, sem = mean_and_stderror_of_mean(
    data=sp,
    dims=(
        "time",
        "cloud_id",
    ),
)


fig, ax = plt.subplots(figsize=(10, 4.5))

ax.plot(
    sp["time"],
    sp.T,
    color="grey",  # [0.75, 0.75, 0.75],
    linestyle="-",
    # marker = ".",
    alpha=0.1,
    zorder=0,
)
ax.plot(
    np.nan,
    np.nan,
    color="grey",  # [0.75, 0.75, 0.75],
    linestyle="-",
    # marker = ".",
    alpha=0.1,
    zorder=0,
    label="Individual clouds",
)

ax.plot(
    precip_cloud_mean["time"],
    precip_cloud_mean.T,
    color="k",  # [0.75, 0.75, 0.75],
    linestyle="-",
    alpha=1,
    zorder=3,
    label="Mean over clouds",
)

ax.fill_between(
    precip_cloud_mean["time"],
    (precip_cloud_mean - 2 * precip_cloud_sem).T,
    (precip_cloud_mean + 2 * precip_cloud_sem).T,
    color="k",
    alpha=0.5,
    zorder=2,
    label="2 SEM",
)

ax.hlines(
    m,
    0,
    3600,
    color="darkorange",
    linestyle="-",
    linewidth=1,
    alpha=1,
    zorder=4,
    label="Total mean",
)

ax.fill_between(
    [0, 3600],
    [m - 2 * sem, m - 2 * sem],
    [m + 2 * sem, m + 2 * sem],
    color="orange",
    alpha=0.5,
    zorder=3,
    label="2 SEM",
)

cloud_ids = [18, 142]
for color, cloud_id in zip(("r", "b"), cloud_ids):
    ax.plot(
        sp["time"],
        sp.sel(cloud_id=cloud_id).T,
        linestyle="-",
        linewidth=0.3,
        marker=".",
        markersize=10,
        color=color,
        alpha=0.75,
        zorder=2,
        label=f"Cloud {cloud_id}",
    )


ax.set_xlim(sp["time"].min(), sp["time"].max())
ax.set_ylim(0, 0.5)

ax.set_xlabel("Time [s]")
ax.set_ylabel(label_from_attrs(sp))

ax.legend(loc="upper left", handler_map=handler_map_alpha())

fig.savefig(
    bbox_inches="tight", fname=fig_dir / f"precipitation_temporal_start_and_individual_clouds.svg"
)
fig.savefig(
    bbox_inches="tight", fname=fig_dir / f"precipitation_temporal_start_and_individual_clouds.pdf"
)

# Influence of radius bin on precipitation

In [ ]:
m, sem = mean_and_stderror_of_mean(
    data=data["precipitation"].sel(time=time_slice).fillna(0),
    dims=("time",),
)
m = m.compute()
m.attrs.update(**data["precipitation"].attrs)

fig, ax = plt.subplots(figsize=(5, 5))

# get the inferno cmap and make the below values transparent
cmap = plt.get_cmap("inferno")
cmap.set_under(color=[0, 0, 0, 0])

m = m.sortby(m.sum(dim="radius_bins"), ascending=False)

pcm = ax.pcolormesh(
    m["radius_bins"],
    m["cloud_id"].astype(str),
    m,
    shading="nearest",
    cmap=cmap,
    # norm = mcolors.LogNorm(),
    vmin=1e-12,
    vmax=None,
)

fig.colorbar(pcm, ax=ax, label=label_from_attrs(m), extend="both")
ax.set_xscale("log")
ax.set_xlim(3, None)
ax.set_yticks([])

ax.set_xlabel(label_from_attrs(m["radius_bins"]))
ax.set_ylabel("Individual clouds")

fig.tight_layout()
fig.savefig(bbox_inches="tight", fname=fig_dir / f"precipitation_radius_bins.svg")
fig.savefig(bbox_inches="tight", fname=fig_dir / f"precipitation_radius_bins.pdf")

# Correlation precipitation with LWC

In [ ]:
lwc = (
    data["mass_represented_per_volume"]
    .sel(gridbox=data["max_gridbox"])
    .sum("radius_bins", keep_attrs=True)
)
lwc = lwc.compute()
lwc.attrs["units"] = "g m^{-3}"
lwc.attrs["long_name"] = "Liquid water content"


lwc_time_mean, lwc_time_sem = mean_and_stderror_of_mean(
    data=lwc,
    dims=("time",),
)

lwc_time_mean = lwc_time_mean.compute()
lwc_time_mean.attrs["units"] = "g m^{-3}"
lwc_time_mean.attrs["long_name"] = "Liquid water content"
lwc_time_sem = lwc_time_sem.compute()

# lwc_cloud_mean, lwc_cloud_sem = mean_and_stderror_of_mean(
#     data=lwc_time_mean,
#     dims=("cloud_id",),
#     data_std=None,
# )

# lwc_cloud_mean = lwc_cloud_mean.compute()
# lwc_cloud_sem = lwc_cloud_sem.compute()

In [ ]:
surface_prec_time_mean, surface_prec_time_sem = mean_and_stderror_of_mean(
    data=surface_prec,
    dims=("time",),
)

surface_prec_time_mean = surface_prec_time_mean.compute()
surface_prec_time_mean.attrs.update(**surface_prec.attrs)
surface_prec_time_sem = surface_prec_time_sem.compute()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4.5))

ax.errorbar(
    x=lwc_time_mean,
    y=surface_prec_time_mean,
    xerr=lwc_time_sem,
    yerr=surface_prec_time_sem,
    marker=".",
    linestyle="",
    label="Individual cloud",
)

linear_fit_plot(
    ax,
    lwc_time_mean,
    surface_prec_time_mean,
)

ax.set_xlabel(label_from_attrs(lwc_time_mean))
ax.set_ylabel(label_from_attrs(surface_prec_time_mean))
ax.set_xlim([0, 0.17])
ax.set_xticks([0, 0.05, 0.1, 0.15])
ax.set_ylim([0, 0.2])
ax.set_yticks([0, 0.05, 0.1, 0.15, 0.2])
ax.legend(loc="upper left")
fig.tight_layout()
fig.savefig(bbox_inches="tight", fname=fig_dir / "scatter-precipitation-liquid_water_content.svg")
fig.savefig(bbox_inches="tight", fname=fig_dir / "scatter-precipitation-liquid_water_content.pdf")

# Stationary state 

In [ ]:
# select random cloud id from the dataset
np.random.seed(42)
cloud_id = np.random.choice(condensation["cloud_id"].values)
print(cloud_id)

my_cmap = mpl.colormaps["cividis"].resampled(15)
# my_cmap.set_over("k")
my_cmap.set_under([1, 1, 1, 0])
my_norm = mcolors.Normalize(vmin=1e-12, vmax=30)


ds = condensation.sel(gridbox=0).sel(cloud_id=cloud_id)
da = ds["xi_per_volume"].compute()


fig, ax = plt.subplots(
    figsize=(8, 4),
)

altitude = ds["gridbox_coord3"]
pcm = ax.pcolormesh(
    da["time"],
    da["radius_bins"],
    da.T,
    shading="auto",
    cmap=my_cmap,
    norm=my_norm,
)
ax.text(
    50,
    1e0,
    f'Gridbox {ds["gridbox"].values} at {altitude.values}m\nCloud {ds["cloud_id"].values}',
    horizontalalignment="left",
    verticalalignment="top",
)

ax.set_ylabel(label_from_attrs(da["radius_bins"]))
ax.set_yscale("log")
ax.set_yticks([1e0, 1e1, 1e2, 1e3])
ax.set_ylim(1e0, 1e3)
ax.invert_yaxis()

ax.set_xlim(0, 3600)


fig.colorbar(
    ax=ax,
    mappable=pcm,
    orientation="vertical",
    label=label_from_attrs(da, units_appendix=" (\\log\\mu m)^{-1}"),
    extend="both",
)


ax.vlines(
    x=(time_slice.start, time_slice.stop - 15),
    ymin=1,
    ymax=1e4,
    color="w",
    linestyle="-",
    linewidth=4,
    zorder=5,
    alpha=0.7,
)
ax.vlines(
    x=(time_slice.start, time_slice.stop - 15),
    ymin=1,
    ymax=1e4,
    color="r",
    linewidth=3,
    linestyle="--",
    label="Stationary state",
    zorder=6,
)

ax.set_xlim(0, time_slice.stop)

ax.set_xlabel("Time [s]")
ax.set_ylabel(label_from_attrs(sp))

ax.legend(handler_map=handler_map_alpha())

fig.tight_layout()
fig.savefig(bbox_inches="tight", fname=fig_dir / "multiplicity_time_radius_bins.pdf")

295


In [ ]:
# select random cloud id from the dataset
np.random.seed(42)
cloud_id = np.random.choice(condensation["cloud_id"].values)
print(cloud_id)

my_cmap = mpl.colormaps["cividis"].resampled(15)
# my_cmap.set_over("k")
my_cmap.set_under([1, 1, 1, 0])
my_norm = mcolors.Normalize(vmin=1e-12, vmax=30)


ds = coalbure_condensation_small.sel(gridbox=0).sel(cloud_id=cloud_id)
da = ds["xi_per_volume"].compute()


fig, ax = plt.subplots(
    figsize=(8, 4),
)

altitude = ds["gridbox_coord3"]
pcm = ax.pcolormesh(
    da["time"],
    da["radius_bins"],
    da.T,
    shading="auto",
    cmap=my_cmap,
    norm=my_norm,
)
ax.text(
    50,
    1e0,
    f'Gridbox {ds["gridbox"].values} at {altitude.values}m\nCloud {ds["cloud_id"].values}',
    horizontalalignment="left",
    verticalalignment="top",
)

ax.set_ylabel(label_from_attrs(da["radius_bins"]))
ax.set_yscale("log")
ax.set_yticks([1e0, 1e1, 1e2, 1e3])
ax.set_ylim(1e0, 1e3)
ax.invert_yaxis()

ax.set_xlim(0, 3600)


fig.colorbar(
    ax=ax,
    mappable=pcm,
    orientation="vertical",
    label=label_from_attrs(da, units_appendix=" (\\log\\mu m)^{-1}"),
    extend="both",
)

ax.vlines(
    x=(time_slice.start, time_slice.stop - 15),
    ymin=1,
    ymax=1e4,
    color="w",
    linestyle="-",
    linewidth=4,
    zorder=5,
    alpha=0.7,
)
ax.vlines(
    x=(time_slice.start, time_slice.stop - 15),
    ymin=1,
    ymax=1e4,
    color="r",
    linewidth=3,
    linestyle="--",
    label="Stationary state",
    zorder=6,
)

ax.set_xlim(0, time_slice.stop)

ax.set_xlabel("Time [s]")
ax.set_ylabel(label_from_attrs(sp))

ax.legend(handler_map=handler_map_alpha())

fig.tight_layout()
fig.savefig(bbox_inches="tight", fname=fig_dir / "multiplicity_time_radius_bins-cbr_small.pdf")

295


In [ ]:
my_cmap = mpl.colormaps["plasma"].resampled(15)
# my_cmap.set_over("k")
my_cmap.set_under([1, 1, 1, 0])
my_norm = mcolors.Normalize(vmin=1e-12, vmax=30)


ds = condensation.sel(cloud_id=cloud_id)
ds = ds.sel(gridbox=ds["max_gridbox"] - 1)
da = ds["xi_per_volume"].compute()

fig, ax = plt.subplots(
    figsize=(8, 4),
)

altitude = ds["gridbox_coord3"]
pcm = ax.pcolormesh(
    da["time"],
    da["radius_bins"],
    da.T,
    shading="auto",
    cmap=my_cmap,
    norm=my_norm,
)
ax.text(
    50,
    1e0,
    f'Gridbox {ds["gridbox"].values} at {altitude.values}m\nCloud {ds["cloud_id"].values}',
    horizontalalignment="left",
    verticalalignment="top",
)

ax.set_ylabel(label_from_attrs(da["radius_bins"]))
ax.set_yscale("log")
ax.set_yticks([1e0, 1e1, 1e2, 1e3])
ax.set_ylim(1e0, 1e3)
ax.invert_yaxis()

ax.set_xlabel("Time $s$")
ax.set_xlim(0, 3600)


fig.colorbar(
    ax=ax,
    mappable=pcm,
    orientation="vertical",
    label=label_from_attrs(da, units_appendix=" (\\log\\mu m)^{-1}"),
    extend="both",
)

fig.tight_layout()
fig.savefig(bbox_inches="tight", fname=fig_dir / "multiplicity_time_radius_bins_cloud_base.pdf")

In [ ]:
fig, ax = plt.subplots(
    figsize=(10, 5),
)

my_cmap = mpl.colormaps["Greys_r"].resampled(15)
# my_cmap.set_over("k")
my_cmap.set_under([1, 1, 1, 0])
my_norm = mcolors.Normalize(vmin=1e-12, vmax=50)

da = condensation["xi_per_volume"].sel(gridbox=0).sel(cloud_id=cloud_id).compute()


altitude = ds["gridbox_coord3"]
pcm = ax.pcolormesh(
    da["time"],
    da["radius_bins"],
    da.T,
    shading="auto",
    alpha=0.5,
    cmap=my_cmap,
    norm=my_norm,
)
ax.set_title(f"Gridbox {ds['gridbox'].values} at {altitude.values}m")
ax.set_yscale("log")
ax.set_yticks([1e0, 1e1, 1e2, 1e3])
ax.set_ylim(1e-1, 1e3)
ax.invert_yaxis()
ax.set_xlim(0, 3600)
ax.set_xlabel("Time $s$")


fig.colorbar(
    ax=ax,
    mappable=pcm,
    orientation="vertical",
    label=label_from_attrs(da, units_appendix=" (\\log\\mu m)^{-1}"),
    extend="both",
)

my_cmap = mpl.colormaps["cool_r"].resampled(15)
# my_cmap.set_over("k")
my_cmap.set_under([1, 1, 1, 0])
my_norm = mcolors.Normalize(vmin=1e-12, vmax=0.1)


ds = condensation.sel(gridbox=0).sel(cloud_id=142)
da = ds["mass_left"].compute()

altitude = ds["gridbox_coord3"]
pcm = ax.pcolormesh(
    da["time"],
    da["radius_bins"],
    da.T,
    shading="auto",
    cmap=my_cmap,
    norm=my_norm,
)
ax.set_title(f"Gridbox {ds['gridbox'].values} at {altitude.values}m")
ax.set_yscale("log")
ax.set_yticks([1e0, 1e1, 1e2, 1e3])
ax.set_ylim(1e0, 1e3)
ax.invert_yaxis()
ax.set_xlim(0, 3600)
ax.set_xlabel("Time $s$")


fig.colorbar(
    ax=ax,
    mappable=pcm,
    orientation="vertical",
    label=label_from_attrs(da, units_appendix=" (\\log\\mu m)^{-1}"),
    extend="both",
)

fig.tight_layout()
fig.savefig(bbox_inches="tight", fname=fig_dir / "multiplicity_time_radius_bins_xi_mass_left.pdf")

In [ ]:
(coalbure_condensation_large["number_superdroplets_per_volume"] * 8).sel(cloud_id=142).isel(
    radius_bins=9
).plot()
plt.figure()
(coalbure_condensation_large["number_superdroplets_per_volume"] * 8).sel(cloud_id=142).isel(
    radius_bins=10
).plot()
# plt.imshow(condensation['number_superdroplets_per_volume'].sel(cloud_id=142).mean("time"))

# Comparison of LWC

In [ ]:
cloud_composite = xr.open_dataset(
    "/home/m/m301096/repositories/sdm-eurec4a/data/observation/cloud_composite/processed/cloud_composite_si_units.nc"
)
identified_clouds = xr.open_dataset(
    "/home/m/m301096/repositories/sdm-eurec4a/data/observation/cloud_composite/processed/identified_clusters/identified_clusters_rain_mask_5.nc"
)

In [ ]:
from sdm_eurec4a.identifications import match_clouds_and_cloudcomposite, select_individual_cloud_by_id


# match_clouds_and_cloudcomposite(
#     ds_clouds = identified_clouds,
#     ds_cloudcomposite = cloud_composite,
#     )

In [ ]:
lwc_means = []
lwc_sems = []
lwc_datas = dict()

for cloud_id in condensation["cloud_id"]:
    da = select_individual_cloud_by_id(identified_clouds, cloud_id)
    start = da["start"].values[0]
    end = da["end"].values[0]
    ds_match = cloud_composite.sel(time=slice(start, end))
    lwc = ds_match["liquid_water_content_original"]
    lwc_mean, lwc_sem = mean_and_stderror_of_mean(
        data=lwc,
        dims=("time",),
    )
    lwc_mean = lwc_mean.compute()
    lwc_sem = lwc_sem.compute()

    lwc_mean = lwc_mean.expand_dims(dim=dict(cloud_id=[cloud_id]))
    lwc_sem = lwc_sem.expand_dims(dim=dict(cloud_id=[cloud_id]))
    lwc = lwc.expand_dims(dim=dict(cloud_id=[cloud_id])).drop("time")

    lwc_means.append(lwc_mean)
    lwc_sems.append(lwc_sem)
    lwc_datas[str(cloud_id.values)] = lwc

lwc_mean = xr.concat(lwc_means, dim="cloud_id")
lwc_sem = xr.concat(lwc_sems, dim="cloud_id")

In [ ]:
cleo_lwc = 1e3 * null_microphysics["liquid_water_content"].sel(
    gridbox=condensation["max_gridbox"] - 2
).sel(time=time_slice).sum("radius_bins", keep_attrs=True)
cleo_lwc_cloudlayer = 1e3 * null_microphysics["liquid_water_content"].sel(
    gridbox=condensation["max_gridbox"]
).sel(time=time_slice).sum("radius_bins", keep_attrs=True)


cleo_lwc_mean, cleo_lwc_sem = mean_and_stderror_of_mean(
    data=cleo_lwc,
    dims=("time",),
)
cleo_lwc_mean = cleo_lwc_mean.compute()
cleo_lwc_sem = cleo_lwc_sem.compute()

cleo_lwc_mean.attrs.update(units="$g m^{-3}$", long_name="Model cloud base LWC")

cleo_lwc_mean_cloudlayer, cleo_lwc_sem_cloudlayer = mean_and_stderror_of_mean(
    data=cleo_lwc_cloudlayer,
    dims=("time",),
)
cleo_lwc_mean_cloudlayer = cleo_lwc_mean_cloudlayer.compute()
cleo_lwc_sem_cloudlayer = cleo_lwc_sem_cloudlayer.compute()

cleo_lwc_mean_cloudlayer.attrs.update(units="$g m^{-3}$", long_name="Model cloud layer LWC")


cloud_composite_lwc_mean, cloud_composite_lwc_sem = lwc_mean, lwc_sem
cloud_composite_lwc_mean.attrs.update(units="$g m^{-3}$", long_name="ATR observations cloud base LWC")

Surface LWC

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.errorbar(
    y=cleo_lwc_mean,
    x=cloud_composite_lwc_mean,
    yerr=2 * cleo_lwc_sem,
    xerr=2 * cloud_composite_lwc_sem,
    linestyle="None",
    marker=".",
    alpha=0.5,
)
ax.plot([0, 3], [0, 3], color="k", linestyle="--")
ax.set_ylabel(label_from_attrs(cleo_lwc_mean))
ax.set_xlabel(label_from_attrs(cloud_composite_lwc_mean))
# fig.suptitle("Liquid water content comparison\nbetween CLEO and ATR")
ax.set_xlim(0, 3)
ax.set_ylim(0, 3)

fig.tight_layout()
fig.savefig(bbox_inches="tight", fname=fig_dir / "lwc_comparison_cleo_atr.pdf")
ax.set_xlim(0, 0.6)
ax.set_ylim(0, 0.3)
ax.set_aspect("equal", adjustable="box")
ax.set_yticks([0, 0.1, 0.2, 0.3])
ax.set_ylabel(label_from_attrs(cleo_lwc_mean, name_width=18))
ax.set_xlabel(label_from_attrs(cloud_composite_lwc_mean))
fig.tight_layout()
fig.savefig(bbox_inches="tight", fname=fig_dir / "lwc_comparison_cleo_atr_zoom.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.errorbar(
    x=cleo_lwc_mean_cloudlayer,
    xerr=2 * cleo_lwc_sem_cloudlayer,
    y=cleo_lwc_mean,
    yerr=2 * cleo_lwc_sem,
    linestyle="None",
    marker=".",
    alpha=0.5,
)
ax.plot([0, 3], [0, 3], color="k", linestyle="--")
ax.set_ylabel(label_from_attrs(cleo_lwc_mean))
ax.set_xlabel(label_from_attrs(cloud_composite_lwc_mean))
# fig.suptitle("Liquid water content comparison\nbetween CLEO and ATR")
ax.set_xlim(0, 3)
ax.set_ylim(0, 3)

fig.tight_layout()
fig.savefig(bbox_inches="tight", fname=fig_dir / "lwc_comparison_cleo_atr.pdf")
ax.set_xlim(0, 0.6)
ax.set_ylim(0, 0.3)
ax.set_aspect("equal", adjustable="box")
ax.set_yticks([0, 0.1, 0.2, 0.3])
ax.set_ylabel(label_from_attrs(cleo_lwc_mean, name_width=18))
ax.set_xlabel(label_from_attrs(cloud_composite_lwc_mean))
fig.tight_layout()
# fig.savefig(bbox_inches= "tight", fname= fig_dir  / "lwc_comparison_cleo_atr_zoom.pdf")

In [ ]:
psd_datas = dict()

for cloud_id in null_microphysics["cloud_id"]:
    da = select_individual_cloud_by_id(identified_clouds, cloud_id)
    start = da["start"].values[0]
    end = da["end"].values[0]
    ds_match = cloud_composite.sel(time=slice(start, end))
    psd = ds_match["particle_size_distribution"]
    psd = psd.expand_dims(dim=dict(cloud_id=[cloud_id])).drop("time")

    psd_datas[str(cloud_id.values)] = psd

cleo_psd_mean = (
    condensation["xi_per_volume"]
    .sel(gridbox=condensation["max_gridbox"])
    .sel(time=time_slice)
    .mean("time", keep_attrs=True)
    .compute()
)
cleo_psd_init = (
    condensation["xi_per_volume"].sel(gridbox=condensation["max_gridbox"]).sel(time=0).compute()
)
cleo_psd_init_nmp = (
    null_microphysics["xi_per_volume"]
    .sel(gridbox=null_microphysics["max_gridbox"])
    .sel(time=0)
    .compute()
)

In [ ]:
from sdm_eurec4a.input_processing import transfer
import lmfit
from sdm_eurec4a.reductions import shape_dim_as_dataarray


def fit_particle_size_distribution(
    ds_cloudcomposite: xr.Dataset,
    particle_split_radius: float = 45e-6,  # 45 micrometer
) -> transfer.PSD_LnNormal:
    """
    Fits the particle size distribution (PSD) of cloud and rain droplets
    idependently.

    Note
    ----
    The PSD is fitted with a bimodal Lognormal distribution.
    For the cloud droplets, the PSD is fitted with
    - geometric mean between 0.1 micrometer and the split radius.
    - geometric sigma between 0 and 1.7.
    For the rain droplets, the PSD is fitted with
    - geometric mean within the range of radius values provided.

    Parameters
    ----------
    ds_cloudcomposite : xr.Dataset
        Dataset containing the cloud composite data.
    particle_split_radius : float, optional
        The radius at which to split the data into cloud and rain droplets. Default is 45 micrometers.

    Returns
    -------
    psd_fit : transfer.PSD_LnNormal
        The fitted particle size distribution.
    """

    # Split data into cloud and rain
    ds_small_droplets = ds_cloudcomposite.sel(radius=slice(None, particle_split_radius))
    ds_rain_droplets = ds_cloudcomposite.sel(radius=slice(particle_split_radius, None))

    # ======================================
    # Fit the PSDs
    # ======================================

    # Use the PSD_LnNormal model
    psd_rain_fit = transfer.PSD_LnNormal()
    psd_cloud_fit = transfer.PSD_LnNormal()

    # ---------
    # Rain
    # ---------
    data = ds_rain_droplets
    radi2d = shape_dim_as_dataarray(da=data, output_dim="radius")
    psd_model = psd_rain_fit.get_model()

    # update geometric mean to be within range of the data
    psd_rain_fit.update_individual_model_parameters(
        lmfit.Parameter(
            name="geometric_means",
            min=data["radius"].min().data,
            max=data["radius"].max().data,
        )
    )

    # fit model parameters and update them
    model_result = psd_model.fit(
        data=data.data, radii=radi2d.data, params=psd_rain_fit.get_model_parameters(), nan_policy="omit"
    )
    psd_rain_fit.lmfitParameterValues_to_dict(model_result.params)

    # ---------
    # Small cloud and drizzle
    # ---------
    # For this, the parameters need to be updated

    # update geometric mean to be within range of 0.1 micrometer and the split radius
    psd_cloud_fit.update_individual_model_parameters(
        lmfit.Parameter(
            name="geometric_means",
            value=1e-5,
            min=0.1e-6,  # at least 0.1 micrometer
            max=particle_split_radius,  # at most the split radius (default 45 micrometer)
        )
    )
    # update geometric sigma to be within range of 0 and 1.7.
    # NOTE: No real physical meaning, but it is a good range for the fit
    psd_cloud_fit.update_individual_model_parameters(
        lmfit.Parameter(
            name="geometric_sigmas",
            value=1.1,
            min=0,
            max=1.7,
        )
    )

    data = ds_small_droplets
    radi2d = shape_dim_as_dataarray(da=data, output_dim="radius")
    psd_model = psd_cloud_fit.get_model()

    # fit model parameters and update them
    model_result = psd_model.fit(
        data=data.data, radii=radi2d.data, params=psd_cloud_fit.get_model_parameters(), nan_policy="omit"
    )
    psd_cloud_fit.lmfitParameterValues_to_dict(model_result.params)

    # --------
    # Combine the fits
    # --------

    psd_fit = psd_rain_fit + psd_cloud_fit

    return psd_fit

In [ ]:
from sdm_eurec4a.visulization import ncols_nrows_from_N
from sdm_eurec4a.input_processing.transfer import fit_lnnormal_for_psd, fit_2lnnormal_for_psd
from sdm_eurec4a.conversions import lwc_from_psd


np.random.seed(879345)
np.random.seed(142)
random_cloud_ids = np.random.choice(condensation["cloud_id"], 9, replace=False)

psd_fits = dict()
lwc_fits = dict()
for cloud_id in random_cloud_ids:
    cloud_id_str = str(cloud_id)
    psd = psd_datas[cloud_id_str]
    psd = psd.where(psd != 0, drop=True)
    psd = psd.mean("cloud_id")
    psd_fit = fit_particle_size_distribution(
        ds_cloudcomposite=psd,
    )
    psd_fits[cloud_id_str] = psd_fit

    psd_fit = psd_fits[cloud_id_str].eval_func(psd["radius"])

from sdm_eurec4a.conversions import lwc_from_psd

lwc_cleo_init = 1e3 * lwc_from_psd(
    ds=xr.Dataset(data_vars=dict(particle_size_distribution=cleo_psd_init_nmp)),
    sum_dim="radius_bins",
    scale_name="radius_bins",
    scale_factor=1e-6,
)
lwc_cleo_init.attrs.update(units="$g m^{-3}$", long_name="CLEO Liquid water content")
# calculate LWC
for cloud_id in random_cloud_ids:
    cloud_id_str = str(cloud_id)
    psd = psd_datas[cloud_id_str]
    psd_fit = psd_fits[cloud_id_str].eval_func(psd["radius"])

    lwc_fit = 1e3 * lwc_from_psd(xr.Dataset(data_vars=dict(particle_size_distribution=psd_fit)))
    lwc_fits[cloud_id_str] = lwc_fit

In [ ]:
fig, axs = plt.subplots(
    figsize=(15, 15),
    sharex=True,
    sharey=True,
    **ncols_nrows_from_N(len(random_cloud_ids)),
)

for idx, cloud_id in enumerate(random_cloud_ids):
    cloud_id_str = str(cloud_id)
    cc_psd = psd_datas[cloud_id_str].mean("cloud_id")
    psd_fit = psd_fits[cloud_id_str]

    sel_cleo_psd_mean = cleo_psd_mean.sel(cloud_id=cloud_id)
    sel_cleo_psd_init = cleo_psd_init_nmp.sel(cloud_id=cloud_id)
    sel_cleo_psd_init_nmp = cleo_psd_init_nmp.sel(cloud_id=cloud_id)

    lwc_cleo = lwc_cleo_init.sel(cloud_id=cloud_id)
    lwc_fit = lwc_fits[cloud_id_str]
    lwc_atr = cloud_composite_lwc_mean.sel(cloud_id=cloud_id)

    ax = axs.flatten()[idx]

    ax.plot(
        1e6 * cc_psd["radius"],
        cc_psd,
        marker="+",
        linestyle="None",
        color="grey",
        alpha=0.5,
    )

    ax.plot(
        np.nan,
        np.nan,
        marker="+",
        linestyle="None",
        color="grey",
        alpha=1,
        label=f"ATR data {lwc_atr.values:.2f} $g m^{{-3}}$",
    )

    ax.plot(
        1e6 * cc_psd["radius"],
        psd_fit.eval_func(cc_psd["radius"]),
        linestyle="-",
        color="b",
        label=f"ATR Fit {lwc_fit.values:.2f} $g m^{{-3}}$",
    )

    ax.plot(
        sel_cleo_psd_init["radius_bins"],
        sel_cleo_psd_init,
        marker="x",
        linestyle="-",
        color="r",
        label=f"CLEO init {lwc_cleo.values:.2f} $g m^{{-3}}$",
    )

    # ax.plot(
    #     sel_cleo_psd_mean["radius_bins"],
    #     sel_cleo_psd_mean,
    #     marker = "x",
    #     linestyle = "-",
    #     color = "r",
    #     label = f"CLEO stationary"
    # )

    ax.legend(loc="upper left")
    ax.set_title(f"Cloud ID: {cloud_id}")


ax = axs.flatten()[0]
ax.set_xscale("log")
ax.set_yscale("symlog", linthresh=1e0, linscale=1)
ax.set_ylim(0, 1e8)

for ax in axs[-1, :]:
    ax.set_xlabel("Radius [µm]")

for ax in axs[:, 0]:
    ax.set_ylabel("Number concentration $[m^{-3} (log(\mu m))^{-1}]$")

add_subplotlabel(axs, location="title")

fig.savefig(bbox_inches="tight", fname=fig_dir / "comparison_psd_fit_and_lwc_cleo_atr.svg")
fig.savefig(bbox_inches="tight", fname=fig_dir / "comparison_psd_fit_and_lwc_cleo_atr.pdf")

## LWC for all microphysics

In [ ]:
datas = [
    condensation,
    collision_condensation,
    coalbure_condensation_cke,
    coalbure_condensation_large,
    coalbure_condensation_small,
]

bins = np.arange(-500, 50, 2)

fig, ax = plt.subplots(figsize=(7, 4.5))


for data in datas:
    mp = data.attrs["microphysics_short"]
    print(mp)
    color = colors_dict[mp]

    data = data.sel(time=time_slice)

    lh_sum = (data["latent_cooling_full"] * data["gridbox_thickness"]).sum("gridbox")

    lh_sum.attrs["units"] = "Wm^{-2}"
    lh_sum.attrs["long_name"] = "Column int. latent cooling"

    lh_sum_time_mean, lh_sum_time_sem = mean_and_stderror_of_mean(
        data=lh_sum,
        dims=("time",),
    )
    lh_sum_time_mean = lh_sum_time_mean.compute()
    lh_sum_time_sem = lh_sum_time_sem.compute()

    lh_sum_cloud_mean, lh_sum_cloud_sem = mean_and_stderror_of_mean(
        data=lh_sum_time_mean,
        dims=("cloud_id",),
        data_std=lh_sum_time_sem,
    )
    lh_sum_cloud_mean = lh_sum_cloud_mean.compute()
    lh_sum_cloud_sem = lh_sum_cloud_sem.compute()

    ax.hist(
        lh_sum_time_mean,
        bins=bins,
        color=[0.85, 0.85, 0.85],
        alpha=1,
        density=True,
        zorder=1,
    )
    ax.axvline(
        lh_sum_cloud_mean,
        color=color,
        label="Mean",
        zorder=3,
    )

    ax.fill_betweenx(
        [0, 0.1],
        lh_sum_cloud_mean - 2 * lh_sum_cloud_sem,
        lh_sum_cloud_mean + 2 * lh_sum_cloud_sem,
        color=color,
        alpha=0.1,
        label="2 SEM",
        zorder=2,
    )

    # ax.axvline(
    #     lh_sum_time_mean.compute().median("cloud_id"),
    #     color="red",
    #     label="Median",
    #     linestyle = "--"
    # )


ax.set_xlabel(label_from_attrs(lh_sum))
ax.set_ylabel("Density")
ax.set_title(f"Histogram of {label_from_attrs(lh_sum, return_units=False)}")

ax.set_ylim(0, 0.065)

ax.set_xlim(10, 110)
fig.savefig(
    bbox_inches="tight",
    fname=fig_dir / "histogram_column_integrated_latent_cooling_all_microphysics.svg",
)
fig.savefig(
    bbox_inches="tight",
    fname=fig_dir / "histogram_column_integrated_latent_cooling_all_microphysics.pdf",
)

condensation
collision_condensation
coalbure_condensation_cke
coalbure_condensation_large
coalbure_condensation_small
